# Deps

In [1]:
""" Dependencies import """

# !pip install pytube
# !pip install moviepy
# !pip install pydub


import os
import numpy as np

import librosa

import pydub
import moviepy.editor as mvd
from pytube import YouTube, Playlist

import cv2 as cv
from cv2 import CAP_PROP_FPS as _cv_frame_rate
from cv2 import CAP_PROP_FRAME_WIDTH as _cv_frame_width
from cv2 import CAP_PROP_FRAME_HEIGHT as _cv_frame_height
from cv2 import CAP_PROP_FRAME_COUNT as _cv_frame_count
from cv2 import CAP_PROP_POS_FRAMES as _cv_current_frame_index

from matplotlib import pyplot as plt

# Download playlist and video

In [18]:
class YTRes():
    low_resolutions = ['360p', '240p', '144p']
    mid_resolutions = ['720p', '480p', '360p']
    high_resolution = ['1080p', '720p']

def download_video(video: str or YouTube, resolution: None or str or list = None):
    if type(video) is str:
        youtubeObject = YouTube(video)
    else:
        youtubeObject = video
    
    if not resolution:
        youtubeObject = youtubeObject.streams.get_highest_resolution()
    elif type(resolution) is list:
        for res in resolution:
            yto = youtubeObject.streams.filter(res=res).first
            if yto != None:
                youtubeObject = yto
                break
    else:
        youtubeObject.streams.filter(res=resolution).first()
        if not youtubeObject:
            print(f'Resolution {res} not avaliable')

    try:
        youtubeObject.download()
        print(f"End download {youtubeObject.title}")
    except:
        print("An error has occurred")
    

def download_playlist(playlist: str or Playlist, resolution: None or str or list = None):
    if type(playlist) is str:
        playlitsObject = Playlist(playlist)
    else:
        playlitsObject = playlist
    
    for iv,video in enumerate(playlitsObject.videos):
        print(f'{iv}. --- Start download {video.title}')
        download_video(video)

In [22]:
video_link = [
    'https://youtu.be/D5veVBWNxYw?list=PLBrn4fh0NHXk28UG7iyYSwfSQY-7a_A-R',
    'https://youtu.be/cXNh1ml9B7U'
][0]

playlist_link = [
    'https://www.youtube.com/playlist?list=PLBrn4fh0NHXk28UG7iyYSwfSQY-7a_A-R', # silly simphonies skeletons
][0]

resolution = YTRes.mid_resolutions

In [23]:
download_video(video_link, resolution=resolution)
# download_playlist(playlist_link, resolution=resolution)

An error has occurred


AttributeError: 'function' object has no attribute 'title'

# Handle video

In [2]:
def video_from_folder(folder: str, fullpath=True, ext: str or list = '.mp4'):
    video_files = []
    ext = list(ext) if type(ext) is str else ext
    _ = [video_files.extend(files_from_folder(folder, fullpath, e)) for e in ext]
    return sorted(video_files)

def audio_from_folder(folder: str, fullpath=True, ext: str or list = '.mp3'):
    audio_files = []
    ext = list(ext) if type(ext) is str else ext
    _ = [audio_files.extend(files_from_folder(folder, fullpath, e)) for e in ext]
    return sorted(audio_files)

def files_from_folder(folder: str, fullpath=True, ext: None or str = None):
    return sorted([f'{folder if fullpath else ""}{f}' for f in os.listdir(folder) if (f.endswith(ext) if ext else True)])

def read_video(filename: str):
    # return cv.VideoCapture(filename)
    return mvd.VideoFileClip(filename)

def video_write_audio(video_filename: str, mp3_filename: None or str = None):
    video = read_video(video_filename)
    mp3_filename = f'{video_filename[ : video_filename.index(".")]}.mp3' if mp3_filename==None else mp3_filename
    video.audio.write_audiofile(mp3_filename)

## Extract audio

In [3]:
vpath = 'VideoMP4/SillySimphoniesSkeletons/'

videopaths = video_from_folder(vpath)

_ = [video_write_audio(vp) for vp in videopaths]

chunk:   1%|▏         | 111/8475 [00:00<00:07, 1107.63it/s, now=None]

MoviePy - Writing audio in VideoMP4/SillySimphoniesSkeletons/Betty Boops Halloween Party 1933 720p.mp3


MoviePy - Done.


chunk:   0%|          | 29/6774 [00:00<00:24, 272.64it/s, now=None]

MoviePy - Writing audio in VideoMP4/SillySimphoniesSkeletons/BLiSS - The Skeleton Dance.mp3


...

MoviePy - Writing audio in VideoMP4/SillySimphoniesSkeletons/The Haunted Ship   -  Van Beuren Studios  (1930).mp3


MoviePy - Done.


chunk:   1%|          | 105/11091 [00:00<00:10, 1047.84it/s, now=None]

MoviePy - Writing audio in VideoMP4/SillySimphoniesSkeletons/Wot A Night (Tom and Jerry Cartoon 1931).mp3


MoviePy - Done.


## Split Audio by Frames 

In [3]:
def files_couple(folder: str, fullpath: bool = True, ext_video: str or list = '.mp4', ext_audio: str or list ='.mp3'):
    video_files = video_from_folder(folder, fullpath=fullpath, ext=ext_video)
    audio_files = audio_from_folder(folder, fullpath=fullpath, ext=ext_audio)
    assert(len(video_files)==len(audio_files))
    return [[vf, af] for vf,af in zip(video_files, audio_files)]

def frame_by_index(video: cv.VideoCapture, frame_index: int):
    assert(frame_index >= 0 and frame_index <= video.get(_cv_frame_count))
    video.set(_cv_current_frame_index, frame_index)
    return video.read()[1]

def write_mp3(filename: str, rate: int, audio: np.array):
    channels = 2 if (audio.ndim == 2 and audio.shape[1] == 2) else 1
    y = np.int16(audio)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=rate, sample_width=2, channels=channels)
    song.export(filename, format="mp3", bitrate="320k")

def frame_sounds_couple(videofile: str, audiofile: str, folder_name: str or None = None, out_name_prefix: str or None = None):
    video = cv.VideoCapture(videofile)
    video_rate = video.get(_cv_frame_rate)
    n_frames = int(video.get(_cv_frame_count))
    audio, audio_rate = librosa.load(audiofile)
    sample_per_frame = audio_rate / video_rate

    folder_name = folder_name if folder_name != None else videofile[:videofile.index('.')]
    out_name_prefix = out_name_prefix if out_name_prefix != None else videofile[videofile.rindex('/')+1 if '/' in videofile else 0 : videofile.index('.')]
    
    os.mkdir(folder_name)

    for fi in range(1, n_frames):
        frame = frame_by_index(video, fi)
        w_start, w_end = int((fi-1)*sample_per_frame), int(fi*sample_per_frame)
        audio_interval = audio[w_start : w_end]
        
        filename = f'{folder_name}{"" if folder_name.endswith("/") else "/"}{out_name_prefix}_seg_{str(fi).rjust(6, "0")}'
        frame_filename, audio_filename = f'{filename}.png', f'{filename}.mp3'

        cv.imwrite(frame_filename, frame)
        write_mp3(audio_filename, audio_rate, audio_interval)


In [4]:
fc = files_couple('VideoMP4/SillySimphoniesSkeletons/')

In [5]:
fsc = frame_sounds_couple(*fc[0])

c:\Users\tommaso\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)

